In [16]:
import pandas as pd
from pulp import *

In [17]:
filepath = r"C:\Users\lbetham\GitHub\Fantasy-Premier-League\data"
cleaned_players_df = pd.read_csv(filepath + r"\2022-23\cleaned_players.csv", low_memory=False)
teams_df = pd.read_csv(filepath + r"\2022-23\teams.csv", low_memory=False)
past_seasons_df = pd.read_csv(filepath + r"\cleaned_merged_seasons.csv", low_memory=False)
raw_players_df = pd.read_csv(filepath + r"\2022-23\players_raw.csv", low_memory=False)

In [18]:
model_df = pd.merge(cleaned_players_df,raw_players_df[['first_name','second_name','team_code']],"left",on=['first_name','second_name'])
model_df = pd.merge(model_df,teams_df[['code','name']],"left",left_on='team_code',right_on='code')
model_df.rename({'name':'team_name','code':'team_code'},axis=1,inplace=True)

In [19]:
model_df["full_name"] = model_df["first_name"] + " " + model_df["second_name"]

In [110]:
model_df[model_df["full_name"] == "Gabriel Fernando de Jesus"]

,first_name,second_name,goals_scored,assists,total_points,minutes,goals_conceded,creativity,influence,threat,...,clean_sheets,red_cards,yellow_cards,selected_by_percent,now_cost,element_type,team_code,team_code,team_name,full_name
27,Gabriel,Fernando de Jesus,8,8,120,1871,15,530.9,567.4,1126.0,...,11,0,1,56.7,80,FWD,3,3,Arsenal,Gabriel Fernando de Jesus


In [21]:
# Helper variables
POS = model_df.element_type.unique()
CLUBS = model_df.team_name.unique()




In [113]:
def solve_team(df, fixed_players = None):
    """A function which solves the team optimisation problem"""
    budget = 1000
    pos_available = {'DEF': 5, 'FWD': 3,'GK': 2, 'MID': 5}
    teams_amount = {team: 3 for team in CLUBS}

    # Remove fixed players from the constraints
    if fixed_players is not None:
        for player in fixed_players:
            budget -= df[df["full_name"] == player].now_cost.values[0]
            pos_available[df[df["full_name"]==player].element_type.values[0]] -= 1
            teams_amount[df[df["full_name"]==player].team_name.values[0]] -= 1

        df = df[~df['full_name'].isin(fixed_players)]
    # Initialize Variables
    names = df.full_name.tolist()
    teams = df.team_name.tolist()
    positions = df.element_type.tolist()
    prices = df.now_cost.tolist()
    points = df.total_points.tolist()
    player_ids = [LpVariable("player_" + str(i), cat="Binary") for i in df.index]
    print(f"Budget = {budget} - Positions = {pos_available} - Teams = {teams_amount}")

    prob = LpProblem("FPL_Player_Choices", LpMaximize)
    # Define the objective
    prob += lpSum(player_ids[i] * points[i] for i in range(len(df))) # Objective - maximize points
    # Budget Limit
    prob += lpSum(player_ids[i] * df.now_cost[df.index[i]] for i in range(len(df))) <= budget 
    # Position Limit
    for pos in POS:
        prob += lpSum(player_ids[i] for i in range(len(df)) if positions[i] == pos) <= pos_available[pos] 
    # Club Limit
    for club in CLUBS:
        prob += lpSum(player_ids[i] for i in range(len(df)) if teams[i] == club) <= teams_amount[club] 
    prob.solve()
    return prob

def create_team_df(prob, fixed_players = None):
    """A function which creates a dataframe of the optimal team from the problem solution"""
    team_df = pd.DataFrame(columns=['name','team','position','price','points'])
    for v in prob.variables():
        if v.varValue == 1:
            idx = int(v.name.split("_")[1])
            team_df = team_df.append({'name':names[idx],'team':teams[idx],'position':positions[idx],'price':prices[idx],'points':points[idx]},ignore_index=True)
    if fixed_players is not None:
        for player in fixed_players:
            idx = names.index(player)
            team_df = team_df.append({'name':names[idx],'team':teams[idx],'position':positions[idx],'price':prices[idx],'points':points[idx]},ignore_index=True)
    return team_df

def price_and_points_top_x(model_df, x):
    """A function which returns the price and points of the top x players"""
    top_x = model_df.sort_values(by='points',ascending=False).head(x)
    return top_x.price.sum(), top_x.points.sum()



In [114]:
fixed_players = ['Mohamed Salah',  'Reece James', 'David Raya Martin', 'Declan Rice', 'Gabriel Fernando de Jesus']
solved_prob = solve_team(model_df, fixed_players)
df = create_team_df(solved_prob, fixed_players)
df.sort_values(by=['position','points'], ascending=False)

Budget = 635 - Positions = {'DEF': 4, 'FWD': 2, 'GK': 1, 'MID': 3} - Teams = {'Arsenal': 2, 'Aston Villa': 3, 'Bournemouth': 3, 'Brentford': 2, 'Brighton': 3, 'Chelsea': 2, 'Crystal Palace': 3, 'Everton': 3, 'Fulham': 3, 'Leicester': 3, 'Leeds': 3, 'Liverpool': 2, 'Spurs': 3, 'Man City': 3, 'Man Utd': 3, 'Newcastle': 3, "Nott'm Forest": 3, 'Southampton': 3, 'West Ham': 2, 'Wolves': 3}


,name,team,position,price,points
9,Mohamed Salah,Liverpool,MID,130,265
6,Son Heung-min,Spurs,MID,120,258
7,Jarrod Bowen,West Ham,MID,85,206
1,Conor Gallagher,Chelsea,MID,60,140
12,Declan Rice,West Ham,MID,50,95
2,Alisson Ramses Becker,Liverpool,GK,55,176
11,David Raya Martin,Brentford,GK,45,95
8,Ivan Toney,Brentford,FWD,70,139
13,Gabriel Fernando de Jesus,Arsenal,FWD,80,120
3,Trent Alexander-Arnold,Liverpool,DEF,75,208


In [105]:
price_and_points_top_x(df, 11)

(745, 1941)

In [106]:
price_and_points_top_x(df, 15)

(1000, 2485)